In [60]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"


# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,424 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http:

In [61]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-19 18:37:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-08-19 18:37:43 (11.7 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [62]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [63]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

### Create DataFrames to match tables

### TRANSFORM ###

In [64]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [65]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

#customers_df.filter(customers_df["customer_id"] == '11160821').show()



+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11160821|             1|
|    1099354|             1|
|   37200466|             1|
|   31581262|             1|
|   27596904|             1|
|    2405598|             1|
|   11229098|             1|
|   17237226|             1|
|   46000147|             1|
|   15234640|             1|
|   28937218|             1|
|   43920023|             1|
|   27954895|             1|
|    8269764|             1|
|   31689770|             1|
|   30117841|             1|
|   38591899|             1|
|   24557051|             1|
|    2780908|             1|
|   32466409|             1|
+-----------+--------------+
only showing top 20 rows



In [66]:
# Create the products_table DataFrame and drop duplicates. 
# product table : product id and product title 

# products_df = df.select([]).drop_duplicates()
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00J7G8EL0|GUM Expanding Flo...|
|B000BGI4DS|MedWeigh MS-2510 ...|
|B00CADD4R6|Cigarette Injecto...|
|B008LEPP3O|Cytomax Powder-Or...|
|B00NTRVVA4|Beauty Wig World ...|
|B000KE8ASK|Inc Magnetic Peda...|
|B00U11YXD8|SG Rubber Foam Mu...|
|B008Y7EZMU|"ABC Products" - ...|
|B00075VVYC|Reizen RL-350 Bra...|
|B005P2KHB0|AUTHENTIC - DESIG...|
|B0002UBEGW|SASSOON HAIR TRIM...|
|B00BHEO7RK|Recliner Risers S...|
|B00GIOTGPW|Hot Tools Dryer -...|
|B00GIX9GLW|Bundle - Doc John...|
|B00GOTNA3A|Wet Light Liquid ...|
|B003VJETUO|BooJee 6 Pk Booje...|
|B003BIFRFQ|Pjur Backdoor Rel...|
|B00HHP2QQ2|ULTRA DNA TM 50 S...|
|B001QGMXAW|Drive Medical Car...|
|B0021YY0YK|Medline ORT27200 ...|
+----------+--------------------+
only showing top 20 rows



In [72]:
# Create the review_id_table DataFrame. 
# review_id_table : review id, customer id, product id, product parent, review date 

# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df.show()

#review_id_df.filter(review_id_df["review_id"] == 'R1QX6706ZWJ1P5').show()



+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1QX6706ZWJ1P5|   32114233|B00OYRW4UE|     223980852| 2015-08-31|
|R3QWMLJHIW6P37|   18125776|B0000537JQ|     819771537| 2015-08-31|
|R14Z1VR1N0Z9G6|   19917519|B00HXXO332|     849307176| 2015-08-31|
| R25ZRJL0GH0U0|   18277171|B00EOB0JA2|     700864740| 2015-08-31|
|R3837KYH7AZNIY|    2593270|B00OC2O1UC|     794298839| 2015-08-31|
|R2MN0QYCY6EVIV|    2592955|B00HES9CMS|     318730927| 2015-08-31|
|R3AN2UJ1D42ZS0|   15168265|B0016BFR4G|     887476137| 2015-08-31|
|R3U29ZLUWEEK4M|   13761624|B00K504UUG|     458706868| 2015-08-31|
|R16ZDMJJHK796C|   37070734|B00HES9CMS|     318730927| 2015-08-31|
| RRRDOEJZD1Y22|   29615023|B00P6TUO5G|     170248843| 2015-08-31|
|R2KR5ZEAS859DK|   47893062|B0006VJ6TO|     412568457| 2015-08-31|
| RR7PGQY763IHF|    2582596|B00H9L7VIW|     851045898| 2015-08

In [68]:
# Create the vine_table. DataFrame
#vine_table : review id, star rating, helpful votes, total votes, vine, verified purchase 
 
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|                Y|
|R3QWMLJHIW6P37|          5|            0|          0|   N|                Y|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|                Y|
| R25ZRJL0GH0U0|          2|            0|          0|   N|                Y|
|R3837KYH7AZNIY|          4|            0|          1|   N|                Y|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|                Y|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|                Y|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|                Y|
|R16ZDMJJHK796C|          5|            0|          0|   N|                N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [69]:
############### LOAD ##################

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonvineanalysis.cspx06bmiwjc.us-west-1.rds.amazonaws.com:5432/amazonvineanalysis"
config = {"user":"postgres", 
          "password": "EEe2991sh12$", 
          "driver":"org.postgresql.Driver"}

In [73]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)